In [14]:
import os
os.environ["PYTHONNET_RUNTIME"] = "coreclr"
os.environ["DOTNET_SYSTEM_DRAWING_USE_GDIPLUS"] = "1"

import clr
import numpy as np
import time
from datetime import timedelta
from pythonnet import load
from System import Environment, Array, String

# Load .NET Core runtime
load("coreclr")

# Try to import System.IO; fallback to os.chdir if it fails
try:
    from System.IO import Directory, Path, File
    use_dotnet_dir = True
except Exception as e:
    print(f"Note: System.IO import failed ({e}), using os.chdir instead")
    import os
    use_dotnet_dir = False

# Define DWSIM path
dwsimpath = "/usr/local/lib/dwsim/"

# Set working directory
if use_dotnet_dir:
    Directory.SetCurrentDirectory(dwsimpath)
else:
    os.chdir(dwsimpath)

# Add DWSIM assemblies
clr.AddReference(dwsimpath + "CapeOpen.dll")
clr.AddReference(dwsimpath + "DWSIM.Automation.dll")
clr.AddReference(dwsimpath + "DWSIM.Interfaces.dll")
clr.AddReference(dwsimpath + "DWSIM.GlobalSettings.dll")
clr.AddReference(dwsimpath + "DWSIM.SharedClasses.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.dll")
clr.AddReference(dwsimpath + "DWSIM.UnitOperations.dll")
clr.AddReference(dwsimpath + "DWSIM.Inspector.dll")
clr.AddReference(dwsimpath + "System.Buffers.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.ThermoC.dll")

# Now import DWSIM types
from DWSIM.Interfaces.Enums.GraphicObjects import ObjectType
from DWSIM.Thermodynamics import Streams, PropertyPackages
from DWSIM.UnitOperations import UnitOperations
from DWSIM.Automation import Automation3
from DWSIM.GlobalSettings import Settings

print("DWSIM imports successful!")

DWSIM imports successful!


In [15]:
# Create an instance of the Automation3 class from the DWSIM.Automation module
# This class provides methods for automating tasks in DWSIM, such as creating and manipulating flowsheets
interf = Automation3()

In [16]:
# Creates a flowsheet
sim = interf.CreateFlowsheet()

In [17]:
# Add Compounds
cnames = ["Water"]
sim.AddCompound("Water")

In [18]:
one = sim.AddFlowsheetObject('Material Stream','1')
two = sim.AddFlowsheetObject('Material Stream','2')
three = sim.AddFlowsheetObject('Material Stream','3')
SPL_1 = sim.AddFlowsheetObject('Stream Splitter','SPL-1')

In [19]:
one = one.GetAsObject()
two = two.GetAsObject()
three = three.GetAsObject()
SPL_1 = SPL_1.GetAsObject()

In [20]:
sim.ConnectObjects(one.GraphicObject , SPL_1.GraphicObject, -1, -1)
sim.ConnectObjects(SPL_1.GraphicObject , two.GraphicObject, -1, -1)
sim.ConnectObjects(SPL_1.GraphicObject , three.GraphicObject, -1, -1)

In [21]:
sim.AutoLayout()

In [22]:
one.SetOverallComposition(Array[float]([1]))
one.SetTemperature(350.0) # K
one.SetPressure(101325.0) # Pa
one.SetMassFlow(1.0) # kg/s

'1: mass flow set to 1 kg/s'

In [23]:
# property package
Steam_table = sim.CreateAndAddPropertyPackage("Steam Tables (IAPWS-IF97)")

In [24]:
# Calc Modes
SPL_1.OperationMode.StreamMassFlowSpec
SPL_1.StreamFlowSpec = 0.6
SPL_1.Stream2FlowSpec = 0.4

In [25]:
# request a calculation
errors = interf.CalculateFlowsheet4(sim)
list(errors)

[]

In [26]:
# save file

fileNameToSave = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), "/workspace/00 FlowSheet Automation/01 Splitter Automation/01 Splitter.dwxmz")

interf.SaveFlowsheet(sim, fileNameToSave, True)